In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
fastballDL = pd.read_csv('DraftLeagueFastball.csv')
fastballMLB = pd.read_csv('MLBFastball.csv')

In [3]:
fastballMLB = fastballMLB[fastballMLB['pitches'] > 50]
fastballMLB.drop('pitches', inplace=True, axis=1)

In [4]:
fastballDL['league'] = 'Draft League'
fastballMLB['league'] = 'MLB'

In [5]:
fastballDL = fastballDL.astype(str)
fastballMLB = fastballMLB.astype(str)

In [6]:
pitchersDL = list(fastballDL['pitcher'])

In [7]:
fastball = pd.concat([fastballDL, fastballMLB])

In [8]:
def combined_features(row):
    return row['avgvelo']+" "+row['avgspinrate']+" "+row['avghorzbreak']+" "+row['avgvertbreak']+" "+row['avgeffectvelo']

In [9]:
#fig, ax = plt.subplots(figsize=(100,100))
#sns.heatmap(cosine_sim,cmap='magma',ax=ax)

In [10]:
def get_index_from_pitcher(pitcher):
    return fastball[fastball.pitcher == pitcher].index[0]

In [11]:
def get_pitcher_from_index(index):
    return fastball[fastball.index == index].values[0][0]

In [12]:
mlb_draft_pitcher = []
mlb_comp = []
cos_value = []

for x in pitchersDL:
    
    mlb_draft_pitcher.append(x)
    
    try:

        fastball = pd.concat([fastballDL, fastballMLB])

        fastball["combined_features"] = fastball.apply(combined_features, axis =1)

        cv = CountVectorizer()
        count_matrix = cv.fit_transform(fastball["combined_features"])

        cosine_sim = cosine_similarity(count_matrix)

        pitcher_like = x
        fastball = fastball[(fastball['pitcher'] == pitcher_like) | (fastball['league'] == 'MLB')]

        player_index = get_index_from_pitcher(pitcher_like)

        similar_pitchers = list(enumerate(cosine_sim[player_index]))

        sorted_similar_pitchers = sorted(similar_pitchers, key=lambda x:x[1], reverse=True)

        mlb_comp.append(get_pitcher_from_index(sorted_similar_pitchers[1][0]))
        
        cos_value.append(sorted_similar_pitchers[1][1])
                
    except IndexError:
        mlb_comp.append('No Comp')
        cos_value.append('No Similarity')
        pass

In [16]:
cosine_comp = pd.DataFrame(list(zip(mlb_draft_pitcher, mlb_comp, cos_value)), columns=['MLB DL Pitcher', 'MLB Comp', "Cosine Value"])
cosine_comp = cosine_comp.sort_values('MLB DL Pitcher')
cosine_comp

,MLB DL Pitcher,MLB Comp,Cosine Value
113,"Alba, Max",No Comp,No Similarity
131,"Alba, Miguel","Rainey, Tanner",0.286039
114,"Allain, Luke","Gose, Anthony",0.316228
37,"Amerlaan, Ashe","Garcia, Luis",0.365148
35,"Ammerlaan, Ashe","Nola, Aaron",0.381385
...,...,...,...
44,"Wilson, Benjamin","Feyereisen, J.P.",0.333333
58,"Wilson, Jeffrey","Cease, Dylan",0.333333
40,"Wiseman, Kevin","Pagán, Emilio",0.333333
141,"Wojtarowicz, Andre",No Comp,No Similarity


In [14]:
cosine_comp.to_csv('fastball_cosine_comp.csv', sep='\t')